In [2]:
! unzip ../input/jigsaw-toxic-comment-classification-challenge/train.csv.zip 
! unzip ../input/jigsaw-toxic-comment-classification-challenge/test_labels.csv.zip 
! unzip ../input/jigsaw-toxic-comment-classification-challenge/sample_submission.csv.zip 
! unzip ../input/jigsaw-toxic-comment-classification-challenge/test.csv.zip 

Archive:  ../input/jigsaw-toxic-comment-classification-challenge/train.csv.zip
replace train.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: ^C
Archive:  ../input/jigsaw-toxic-comment-classification-challenge/test_labels.csv.zip
replace test_labels.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: ^C
Archive:  ../input/jigsaw-toxic-comment-classification-challenge/sample_submission.csv.zip
replace sample_submission.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: ^C
Archive:  ../input/jigsaw-toxic-comment-classification-challenge/test.csv.zip
replace test.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: ^C


In [5]:
! pip install transformers

In [3]:
import numpy as np
import pandas as pd
import re
import tensorflow as tf
import transformers
from tqdm.notebook import tqdm

In [4]:
tokenizer=transformers.BertTokenizer.from_pretrained("bert-base-uncased", do_lower_case=True)
max_length=128

In [5]:
batch_size=32
max_len=128
EPOCHS=2

In [10]:
data=pd.read_csv("./train.csv",nrows=1000)
test_labels=pd.read_csv("./test_labels.csv",nrows=1000)
# train_data.head() 
# print(data.loc[0])
# data.columns

# data["toxic"].value_counts()
data.head()
test_labels.head()

,id,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,00001cee341fdb12,-1,-1,-1,-1,-1,-1
1,0000247867823ef7,-1,-1,-1,-1,-1,-1
2,00013b17ad220c46,-1,-1,-1,-1,-1,-1
3,00017563c3f7919a,-1,-1,-1,-1,-1,-1
4,00017695ad8997eb,-1,-1,-1,-1,-1,-1


In [11]:
row_no=list(data.shape)[0]
for i in range(row_no):
    row=data.loc[i]
    row["comment_text"]=re.sub(r"(@\[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)|^rt|http.+?", "", row["comment_text"])

/opt/conda/lib/python3.7/site-packages/pandas/core/series.py:1056: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cacher_needs_updating = self._check_is_chained_assignment_possible()


In [12]:
# x_train[0]
data.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0


In [32]:
from keras.preprocessing.sequence import pad_sequences
# to convert the iput ids array in same size(column no=max(column no))
tokenizer=transformers.BertTokenizer.from_pretrained("bert-base-uncased", do_lower_case=True)
max_length=128


def tokenize(data,tokenizer=tokenizer,max_length=max_length):
    input_ids=[]
    attention_masks=[]
    for sentence in tqdm(data):
        
        encoded_data=tokenizer.batch_encode_plus(
                        sentence,
                        add_special_tokens=True,
                        max_length=max_length,
                        truncation=True,
                        return_attention_mask=True,
                        return_token_type_ids=True,
                        pad_to_max_length=True,
                        return_tensors="tf",
                    )

        input_id=np.array(encoded_data["input_ids"],dtype="int32")
        attention_mask=np.array(encoded_data["attention_mask"],dtype="int32") 
        
        input_ids.append(input_id)
        attention_masks.append(attention_mask)
        
    return [input_ids,attention_masks]

bert_op=tokenize(data['comment_text'])
input_ids=pad_sequences(bert_op[0],maxlen=max_length,dtype='long',value=0,truncating="post",padding="post")
attention_masks=bert_op[1]

  0%|          | 0/1000 [00:00<?, ?it/s]

In [43]:
attention_masks=np.array(attention_masks)
attention_masks=pad_sequences(attention_masks,maxlen=max_length,dtype='long',value=0,truncating="post",padding="post")
input_ids.shape
attention_masks.shape

(1000, 128, 128)

In [44]:
from sklearn.model_selection import train_test_split
label_cols = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']
labels=data[label_cols].values

train_inputs,validation_inputs,train_labels,validation_labels=train_test_split(input_ids,labels,random_state=0,test_size=0.2)
train_masks,validation_masks,_,_=train_test_split(attention_masks,labels,random_state=0,test_size=0.2)


print(train_inputs.shape)
print(train_labels.shape)

print(validation_inputs.shape)
print(validation_labels.shape)

print(train_masks.shape)

(800, 128, 128)
(800, 6)
(200, 128, 128)
(200, 6)
(800, 128, 128)


In [51]:
# creating batched dataset
epochs=2
def create_batch_dataset(data,epochs=epochs,batch_size=batch_size,buffer_size=1000,train=True):
    dataset=tf.data.Dataset.from_tensor_slices(data)
    if train:
        dataset=dataset.shuffle(buffer_size=buffer_size)
        # uses for shuffling the dataset.Select the first buffer_size element from dataset
    dataset=dataset.repeat(epochs)
    # just repeat the whole dataset
    dataset=dataset.batch(batch_size=batch_size)
    # devide the whole dataset into batch size and create an array of array
    if train:
        dataset=dataset.prefetch(1)
    #     It has no concept of examples vs. batches. examples.prefetch(2) will prefetch two 
    # elements (2 examples), while examples.batch(20).prefetch(2) will prefetch 2 elements (2 
    # batches, of 20 examples each).
    return dataset
train_dataset=create_batch_dataset((train_inputs,train_masks,train_labels),train=True)
validation_dataset=create_batch_dataset((validation_inputs,validation_masks,validation_labels),train=True)

In [52]:
print(train_dataset)
print(validation_dataset)

<PrefetchDataset shapes: ((None, 128, 128), (None, 128, 128), (None, 6)), types: (tf.int64, tf.int64, tf.int64)>
<PrefetchDataset shapes: ((None, 128, 128), (None, 128, 128), (None, 6)), types: (tf.int64, tf.int64, tf.int64)>


In [ ]:
# https://www.kaggle.com/nkaenzig/bert-tensorflow-2-huggingface-transformers